# Redes de Hopfield

Son redes neuronales recurrentes diseñadas para implementar memorias asociativas. 
Estrategia a seguir para estudiar las redes de Hopfield:

- Definir qué es una memoria asociativa.
- Mostrar problemas solucionables utilizando memor.ias asociativas
- Implementación intuitiva de una red de Hopfield.
- Detalle matemático y límites de su funcionamiento.
- Técnicas para ampliar los límites analizados en el punto anterior.

Cabe aclarar que hoy en día las redes de Hopfield no eson muy utilizadas para resolver problemas reales, pero sirven como punto de partida para analizar las Máquinas Restringidas de Boltzmann (RBM, una generalización de las redes de Hopfield). Estudiar las redes de Hopfield nos dará entonces la base necesaria para poder digerir las RBM, así como un buen antipasto auspicia de base inequívoca para digerir un buen plato de pastas.

## Memorias Asociativas

Permite recuperar información a partir de conocimiento parcial de su contenido, sin saber su localización de almacenamiento. A veces también se le llama memoria de direccionamiento por contenido (content-adressable-memory).

Los computadores tradicionales no usan este direccionamiento; se basan en el conocimiento exacto de la dirección de memoria en la que se encuentra la información. Sin embargo, se cree que el cerebro humano no actúa así. Si queremos recordar el nombre de una persona, no nos sirve saber que fue el nombre número 3274 que aprendimos. Es más útil saber que su nombre empieza y termina por 'N' y que es un famoso científico inglés. Con esta información, es casi seguro que recordaremos exitosamente a "Newton".

Cuando la información que ingreso a la memoria asociativa es igual a la información que espero tener a la salida (pero con información faltante o parcialmente incorrecta) se dice que la memoria es autoasociativa.

Ejemplo de memoria autoasociativa:

Supongamos en este ejemplo que las variables de entrada solo pueden valer 1 o -1. La siguiente memoria autoasociativa tiene guardadas cuatro palabras de 7 bits a las que vamos a denominar $\zeta_i$

<img src="images/memo.png" alt="drawing" width=800px/> 

Cada vez que a la entrada de la memoria autoasociativa del ejemplo le asignemos un patrón $\xi$, nuestra memoria nos dará a la salida el patrón $\zeta$ que mas se parezca a nuestro patrón de entrada. Para ello tendremos que haber definido una medida de similitud. En nuestro ejemplo, asumiremos que es la distancia de Hamming,

Nos centraremos en este tipo de memorias (autoasociativas)

### Aplicaciones de memorias autoasociativas

#### OCR (Optical Character Recognition)

Nuestra memoria tiene almacenada un conjunto posible de caracteres. Cada vez que se ingresa a la memoria con un caracter defectuoso, la memoria devuelve el patrón de referencia almacenado que mas se le parezca. 

Ejemplo:

<img src="images/OCR.png" alt="drawing" width=600px/> 

### Sistemas de recomendación 

Para este problema, la hipótesis es que las valoraciones de un usuario sobre un conjunto de productos u obras, es una manifestación ruidosa e incompleta de un patrón de referencia almacenado en la memoria autoasociativa.
Cada bit de la memoria autoasociativa contiene una valoración positiva/negativa de un producto u obra. Por ejemplo:

| Film: | El señor de los anillos | La caza del octubre rojo | La guerra de las galaxias | El acorazado Potemkin | Sex & The city | Lo que ellas quieren | Casablanca | Terminator | Mi primer beso |
|-------------------|:-----------------------:|:------------------------:|:-------------------------:|:---------------------:|:--------------:|:--------------------:|:----------:|:----------:|:--------------:|
| Tipo de usuario 1 | 1 | -1 | 1 | 1 | -1 | -1 | -1 | 1 | -1 |
| Tipo de usuario 2 | 1 | -1 | -1 | -1 | 1 | 1 | 1 | -1 | 1 |
| Tipo de usuario 3 | -1 | 1 | 1 | 1 | -1 | -1 | 1 | -1 | -1 |


En la tabla se ven tres tipos de usuarios ideales. El primer tipo de usuario tiene preferencia por las películas de ciencia ficción y aventuras. El segundo tipo de usuario tiene preferencia por las comedias románticas y el tercer tipo de usuario tiene preferencia por las películas clásicas.
Nótese lo que tiene almacenada la memoria autoasociativa es una idealización de algunos tipos de usuario posible, cada uno con preferencias distintas. Esta representación implica que las valoraciones de un usuario real particular, son una representación ruidosa y con datos faltantes de los gustos que representan los "tipos de usuario" los cuales son ideales.

A partir de los gustos de un usuario real particular que podrían ser (el cero marca que el usuario no ingresó valoración sobre la película):


| Film: | El señor de los anillos | La caza del octubre rojo | La guerra de las galaxias | El acorazado Potemkin | Sex & The city | Lo que ellas quieren | Casablanca | Terminator | Mi primer beso |
|-------------------|:-----------------------:|:------------------------:|:-------------------------:|:---------------------:|:--------------:|:--------------------:|:----------:|:----------:|:--------------:|
| Usuario real y mundano | 1 | 0 | 1 | 0 | -1 | 0 | 0 | 1 | -1 |

La memoria autoasociativa me debería dar a su salida el tipo de usuario que mas se le parezca (En este caso el tipo de usuario 1):


| Film: | El señor de los anillos | La caza del octubre rojo | La guerra de las galaxias | El acorazado Potemkin | Sex & The city | Lo que ellas quieren | Casablanca | Terminator | Mi primer beso |
|-------------------|:-----------------------:|:------------------------:|:-------------------------:|:---------------------:|:--------------:|:--------------------:|:----------:|:----------:|:--------------:|
| Entrada | 1 | 0 | 1 | 0 | -1 | 0 | 0 | 1 | -1 |
| Salida | 1 | -1 | 1 | 1 | -1 | -1 | -1 | 1 | -1 |


Más adelante veremos cómo inferir estos tipos de usuario "ideales" a partir de valoraciones de usuarios reales. Es decir, a partir de las manifestaciones "ruidosas e incompletas".

Estos son algunos ejemplos de referencia de una memoria autoasociativa. Siendo que una forma de implementarlas es mediante las redes de Hopfield, procederemos a estudiarlas.

## Redes de Hopfield: Implementación de una memoria Autoasociativa

**Variables de entrada (bits):** 1 y -1 (o 1 y 0)

**Patrones de entrada:** palabras de N bits 
(Por ser una memoria autoasociativa la **salida** es una palabra de N bits también)

**Definiciones:**
- **𝑁**: 𝑇𝑎𝑚𝑎ñ𝑜 𝑒𝑛 𝑏𝑖𝑡𝑠 𝑑𝑒 𝑙𝑎 𝑒𝑛𝑡𝑟𝑎𝑑𝑎 𝑦 𝑙𝑎 𝑠𝑎𝑙𝑖𝑑𝑎
- **𝜉**:𝑃𝑎𝑡𝑟ó𝑛 𝑑𝑒 𝑒𝑛𝑡𝑟𝑎𝑑𝑎
- **𝜉_𝑖**:𝑏𝑖𝑡 𝑖 𝑑𝑒𝑙 𝑝𝑎𝑡𝑟ó𝑛 𝑑𝑒 𝑒𝑛𝑡𝑟𝑎𝑑𝑎 𝜉
- **𝑆**:𝑃𝑎𝑡𝑟ó𝑛 𝑑𝑒 𝑠𝑎𝑙𝑖𝑑𝑎
- **𝑆_𝑖**:𝑏𝑖𝑡 𝑖 𝑑𝑒𝑙 𝑝𝑎𝑡𝑟ó𝑛 𝑑𝑒 𝑠𝑎𝑙𝑖𝑑𝑎 𝑆
- **𝜍**:𝑃𝑎𝑡𝑟𝑜𝑛𝑒𝑠 𝑑𝑒 𝑟𝑒𝑓𝑒𝑟𝑒𝑛𝑐𝑖𝑎
- **P**: Cantidad de patrones de referencia
- **𝜍^𝜇**:𝑃𝑎𝑡𝑟ó𝑛 𝑑𝑒 𝑟𝑒𝑓𝑒𝑟𝑒𝑛𝑐𝑖𝑎 𝑛ú𝑚𝑒𝑟𝑜 𝜇
- **𝜍_𝑖^𝜇**:𝑏𝑖𝑡 𝑖 𝑑𝑒𝑙 𝑝𝑎𝑡𝑟ó𝑛 𝑑𝑒 𝑟𝑒𝑓𝑒𝑟𝑒𝑛𝑐𝑖𝑎〖 𝜍〗^𝜇

La memoria se puede decir que guarda *Patrones de referencia* a partir de los cuales busca la solución.
![captcha](images/memo.png)

Podemos calcular la salida de la red de Hopfield como:

$$ S_j = sign(\sum_{i=1}^{N} w_{ij} \xi_i) $$ 

![pinv](images/patroneinverso.png)

In [1]:
import numpy as np
from numpy import transpose as t

P = np.array([[1, 1, 1, 1, -1, -1, -1]])
W = P.T@P

print(W)

[[ 1  1  1  1 -1 -1 -1]
 [ 1  1  1  1 -1 -1 -1]
 [ 1  1  1  1 -1 -1 -1]
 [ 1  1  1  1 -1 -1 -1]
 [-1 -1 -1 -1  1  1  1]
 [-1 -1 -1 -1  1  1  1]
 [-1 -1 -1 -1  1  1  1]]


In [3]:
def getReference(P):
    return np.sign(W@P.T).T

In [4]:
def getMatrix(P):
    return P.T@P

In [5]:
Pc = np.array([[1, -1, 1, 1, -1, -1, -1]])
(getReference(Pc) == P).all()

True

In [6]:
Pc = np.array([[-1, -1, -1, 1, -1, 1, 1]])
ref = getReference(Pc)
(ref == P).all()

False

### Almacenamiento de un patrón y su inverso
En sistemas dinámicos un **atractor** es un conjunto de valores numéricos hacia los cuales un sistema tiende a evolucionar a partir de una amplia variedad de condiciones iniciales.

Cuando calculamos la matriz W para que P sea un atractor, -P automáticamente queda definido como atractor también. 

**Cada vez que definimos a P como un patrón de referencia, -P también lo es.**

![captcha](images/atractores.png)

In [7]:
(ref != P).all()  # Para cada elemento, ref[i] != P[i]

True

### Almacenamiento de varios Patrones de Referencia

Cuando se desea almacenar varios patrones de referencia se suele usar:

$ w_{i,j} = \frac{1}{N} \sum_{\mu = 1}^P 𝜍_i^\mu 𝜍_j^\mu$

Ésta ecuación se la denomina *Regla de Hebb* y es el promedio de la matriz de pesos para cada patrón por separado.

Observar:
- Si para distintos patrones de referencia, la correlación entre los bits i y j se mantiene, $𝜔_{𝑖,𝑗}$ tendrá un valor en módulo cercano a 1. 
- Si para distintos patrones de referencia, no hay correlación entre los bits i y j, $𝜔_{𝑖,𝑗}$ tendrá un valor en módulo cercano a 0.



In [8]:
# Defino los patrones de referencia
P1=np.array([[ 1,  1,   1,   1,   1,   1,   1]]);
P2=np.array([[ 1,  1,   1,  -1,  -1,  -1,  -1]]);
P3=np.array([[-1, -1,   1,   1,   1,   1,   1]]);
P4=np.array([[ 1,  1,   1,  -1,  -1,   1,   1]]);
 
# Armo la matriz de pesos
W1 = getMatrix(P1);
W2 = getMatrix(P2);
W3 = getMatrix(P3);
W4 = getMatrix(P4);
 
W = (W1 + W2 + W3 + W4) / 4;

In [9]:
def whichReference(P, ref_mat):
    for i, p in enumerate(ref_mat):
        if (getReference(P) == p).all():
            sign = '-' if i > 3 else ''
            num = str(i%4 + 1)
            print(sign + 'P' + num)

In [10]:
# Calculo la salida para P3 
print((getReference(P3) == P3).all())
 
# Calculo la salida para -P3 
print((getReference(-P3) == -P3).all())
 
Ptot = [P1, P2, P3, P4, -P1, -P2, -P3, -P4]
# Calculo la salida para P2 con error en el bit 4
Pc = np.array([[1, 1, 1, 1, -1, -1, -1]])
whichReference(Pc, Ptot)
        
# Calculo la salida para -P4 con un error en el bit 3
Pc = np.array([[-1, -1, 1, 1, 1, -1, -1]])
whichReference(Pc, Ptot)

True
True
P2
-P4
